In [ ]:
using ForwardDiff
using Symbolics
using RuntimeGeneratedFunctions
using Distributions
using GeometricIntegrators
using Optim
using Plots
using Random
using Distances
RuntimeGeneratedFunctions.init(@__MODULE__)
gr()

In [ ]:
const DEFAULT_LAMBDA = 0.05
const DEFAULT_NOISE_LEVEL = 0.05
const DEFAULT_NLOOPS = 10
const DEFAULT_NOISEGEN_TIMESTEP = 0.01


In [ ]:
struct HamiltonianSINDy{T, GHT}
    # basis::Vector{Symbolics.Num} # the augmented basis for sparsification
    analytical_fθ::GHT
    z::Vector{Symbolics.Num} 
    λ::T # Sparsification Parameter
    noise_level::T # Noise amplitude added to the data
    noiseGen_timeStep::T # Time step for the integrator to get noisy data 
    nloops::Int # Sparsification Loops
    
    function HamiltonianSINDy(
        analytical_fθ::GHT = missing,
        z::Vector{Symbolics.Num} = get_z_vector(2);
        λ::T = DEFAULT_LAMBDA,
        noise_level::T = DEFAULT_NOISE_LEVEL,
        noiseGen_timeStep::T = DEFAULT_NOISEGEN_TIMESTEP,
        nloops = DEFAULT_NLOOPS) where {T, GHT <: Union{Base.Callable,Missing}}

        new{T, GHT}(analytical_fθ, z, λ, noise_level, noiseGen_timeStep, nloops)
    end
end

In [ ]:
struct TrainingData{AT<:AbstractArray}
    x::AT # initial condition
    ẋ::AT # initial condition
    y::AT # noisy data at next time step

    TrainingData(x::AT, ẋ::AT, y::AT) where {AT} = new{AT}(x, ẋ, y)
    TrainingData(x::AT, ẋ::AT) where {AT} = new{AT}(x, ẋ)
end

In [ ]:
abstract type VectorField end

In [ ]:
#The _prod function takes one or more input arrays and performs an element-wise multiplication on them.
_prod(a, b, c, arrs...) = a .* _prod(b, c, arrs...)
_prod(a, b) = a .* b
_prod(a) = a

# generates a vector out of symbolic arrays (p,q) with a certain dimension
function get_z_vector(dims)
    @variables q[1:dims]
    @variables p[1:dims]
    z = vcat(q,p)
    return z
end


# returns the number of required coefficients for the basis
function get_numCoeffs(basis::Vector{Symbolics.Num})
    return length(basis)
end


# gets a vector of combinations of hamiltonian basis
function get_basis_set(basis::Vector{Symbolics.Num}...)
    # gets a vector of combinations of basis
    basis = vcat(basis...)
    
    # removes duplicates
    basis = Vector{Symbolics.Num}(collect(unique(basis)))

    return basis
end

In [ ]:
struct HamiltonianSINDyVectorField{DT,CT,GHT} <: VectorField
    # basis::BT
    coefficients::CT
    fθ::GHT

    function HamiltonianSINDyVectorField(coefficients::CT, fθ::GHT) where {DT, CT <: AbstractVector{DT}, GHT <: Base.Callable}
        new{DT,CT,GHT}(coefficients, fθ)
    end
end


function VectorField(method::HamiltonianSINDy, data::TrainingData; solver = BFGS())
    # Check if the first dimension of x is even
    size(data.x[begin], 1) % 2 == 0 || throw(ArgumentError("The first dimension of x must be even."))

    # dimension of system
    d = size(data.x[begin], 1) ÷ 2
    fθ = ΔH_func_builder_two(d, method.z)
    coeffs = sparsify_extra(method, fθ, data.x, data.ẋ, solver)    
    HamiltonianSINDyVectorField(coeffs, fθ)
end


" wrapper function for generalized SINDY hamiltonian gradient.
Needs the output of fθ to work! "
function (vectorfield::HamiltonianSINDyVectorField)(dz, z)
    vectorfield.fθ(dz, z, vectorfield.coefficients)
    return dz
end

(vectorfield::HamiltonianSINDyVectorField)(dz, z, p, t) = vectorfield(dz, z)

In [ ]:

function ΔH_func_builder_two(d::Int, z::Vector{Symbolics.Num} = get_z_vector(d)) 
    # nd is the total number of dimensions of all the states, e.g. if q,p each of 3 dims, that is 6 dims in total
    # nd = 2d
    
    Dz = Differential.(z)
   
    # gets number of terms in the basis
    @variables a[1:4]

    coeffsMat = Matrix{Symbolics.Num}([a[1] a[2]; a[3] a[4]])
    qs = cos.(coeffsMat * z[1:2])
    ps= 0.5*(transpose(coeffsMat*z[3:4])*(coeffsMat*z[3:4]))
    basis = vcat(qs,ps)
    basis = Vector{Symbolics.Num}(basis)

    # collect and sum combinations of basis and coefficients
    ham = sum(collect(basis))

    # gives derivative of the hamiltonian, but not the skew-symmetric true one
    f = [expand_derivatives(dz(ham)) for dz in Dz]

    #simplify the expression potentially to make it faster
    f = simplify(f)
    
    # line below makes the vector into a hamiltonian vector field by multiplying with the skew-symmetric matrix
    ΔH = vcat(f[d+1:2d], -f[1:d])
    
    # builds a function that calculates Hamiltonian gradient and converts the function to a native Julia function
    ΔH_eval = @RuntimeGeneratedFunction(Symbolics.inject_registered_module_functions(build_function(ΔH, z, a)[2]))
    
    return ΔH_eval
end

In [ ]:
function sparsify_extra(method::HamiltonianSINDy, fθ, x, ẋ, solver)
    # add noise
    ẋnoisy = [_ẋ .+ method.noise_level .* randn(size(_ẋ)) for _ẋ in ẋ]

    coeffs = randn(4)
    println("Initial coeffs:", coeffs)
    
    # define loss function
    function loss_kernel(x₀, x̃, fθ, a)
        # gradient of SINDy Hamiltonian problem
        f = zeros(eltype(a), axes(x₀))
        
        # gradient at current (x) values
        fθ(f, x₀, a)

        # calculate square euclidean distance
        sqeuclidean(f,x̃)
    end

    # define loss function
    function loss(a::AbstractVector)
        mapreduce(z -> loss_kernel(z..., fθ, a), +, zip(x, ẋnoisy))
    end
    
    # initial guess
    println("Initial Guess...")
    result = Optim.optimize(loss, coeffs, solver, Optim.Options(show_trace=true); autodiff = :forward)
    coeffs .= result.minimizer

    println(result)
    println(coeffs)

    for n in 1:method.nloops
        println("Iteration #$n...")

        # find coefficients below λ threshold
        smallinds = abs.(coeffs) .< method.λ
        biginds = .~smallinds

        # check if there are any small coefficients != 0 left
        all(coeffs[smallinds] .== 0) && break

        # set all small coefficients to zero
        coeffs[smallinds] .= 0

        # Regress dynamics onto remaining terms to find sparse coeffs
        function sparseloss(b::AbstractVector)
            c = zeros(eltype(b), axes(coeffs))
            c[biginds] .= b
            loss(c)
        end

        # b is a reference to coeffs[biginds]
        b = coeffs[biginds]
        result = Optim.optimize(sparseloss, b, solver, Optim.Options(show_trace=true); autodiff = :forward)
        b .= result.minimizer

        println(result)
        println(coeffs)
    end

    return coeffs
end

In [ ]:
# --------------------
# Setup
# --------------------

println("Setting up...")

# 2D system with 4 variables [q₁, q₂, p₁, p₂]
const nd = 4
d = nd ÷ 2
z = get_z_vector(Int(nd/2))

# Gradient function of the 2D hamiltonian
grad_H_ana(x) = [x[3]; x[4]; sin(x[1]); sin(x[2])]
function grad_H_ana!(dx, x, p, t)
    dx .= grad_H_ana(x)
end

In [ ]:
println("Generate Training Data...")

# number of samples
num_samp = 25

# samples in p and q space
samp_range = LinRange(-1, 1, num_samp)

# s depend on size of nd (total dims), 4 in the case here so we use samp_range x samp_range x samp_range x samp_range
s = collect(Iterators.product(fill(samp_range, nd)...))

# compute vector field from x state values
x = [collect(s[i]) for i in eachindex(s)]

dx = zeros(nd)
p = 0
t = 0
ẋ = [grad_H_ana!(copy(dx), _x, p, t) for _x in x]

Transform data

In [ ]:
W = [0.5 0.5; 0.5 -0.5]

for i in 1:size(x,1)
    x[i][1:2] .= W * x[i][1:2]
    x[i][3:4] .= W * x[i][3:4] 
    ẋ[i][1:2] .= W * ẋ[i][1:2]
    ẋ[i][3:4] .= W * ẋ[i][3:4]  
end

In [ ]:
# ** noiseGen_timeStep chosen randomly
method = HamiltonianSINDy(grad_H_ana!, z, λ = 0.05, noise_level = 0.0)

# collect training data
tdata = TrainingData(x, ẋ)

In [ ]:
vectorfield = VectorField(method, tdata, solver=BFGS())

Compare gradients of transformed data and SINDy prediction

In [ ]:
ẋ[2]'

In [ ]:
p = 0
t = 0
dx = zero(x[2])
vectorfield(dx, x[2], p, t)'

Compare inverse of W to SINDy predicted coefficients

In [ ]:
inv(W)

transpose reshaped vectorfield.coefficients to get the row major shape for \[\begin{matrix} a[1] & a[2] \\ a[3] & a[4] \end{matrix}\]

In [ ]:
transpose(reshape(vectorfield.coefficients, 2, 2))
